# Ensemble fork with weights

## I forked this notebook: https://www.kaggle.com/code/apoorvbhardwaj/amex-early-ensemble/notebook?scriptVersionId=97132599
## Please give that notebook an upvote if you give this one an upvote...

I took the above notebook and played around with model weights. Pretty simple idea added to the above simple idea.

Here's a good explanation of the idea of doing a weighted ensemble.
https://machinelearningmastery.com/weighted-average-ensemble-with-python/


Anyway, just a slight improvement over the origial linked notebook above
* Original score = 0.794
* This score = 0.794 ++ (the leaderboard needs to add some decimal places...)

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import plotly.express as px

In [2]:
targetName = 'prediciton'
competitionDir = r'C:\Users\Khangjrakpam Arjun\OneDrive - McKinsey & Company\Documents\ARJUN\ML\COMPETITIONS\Amex\amex-default-prediction'
submission = pd.read_csv(r'C:\Users\Khangjrakpam Arjun\OneDrive - McKinsey & Company\Documents\ARJUN\ML\COMPETITIONS\Amex\amex-default-prediction/sample_submission.csv')

In [3]:
import re
regex = "([^\\s]+(\\.(?i)(csv))$)"
p = re.compile(regex)

C:\Users\Khangjrakpam Arjun\AppData\Local\Temp\ipykernel_22444\3298245924.py:3: DeprecationWarning: Flags not at the start of the expression '([^\\s]+(\\.(?i)(csv))' (truncated) but at position 10
  p = re.compile(regex)


In [4]:
preds = []

In [5]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy import stats

# Weighted average
As per the little function below, I wrote a little formula for weighting the predictions that is based on how well the model it came from scored. The best model is weighted 1, and the worst is weighted somewhere around 0.05.

The article that's linked below has some more elegant ways to do the weighting if you actually buld the models youself and can therefore do some cross validation, but here, since I'm just pulling the output from other people's hard work, I can't and can just do the crude little thing below. 
https://machinelearningmastery.com/weighted-average-ensemble-with-python/

In [6]:
#crude formula for model weighting.
# Using this formaula the best model's weight = 1, the worst model's weight ~0.05
def model_weight(model_loss, worst_loss, best_loss):
    return 1- ((best_loss - model_loss)/(best_loss-(worst_loss - 0.01)))

In [7]:
os.chdir(r"C:\Users\Khangjrakpam Arjun\OneDrive - McKinsey & Company\Documents\ARJUN\ML\COMPETITIONS\Amex\amex-default-prediction\submission")

In [8]:
os.listdir()

['AutoML_0.78878.csv',
 'blended',
 'catboost_0.794.csv',
 'keras_0.78326.csv',
 'LGBM_0.795.csv',
 'results.zip',
 'XGBoost_0.7776.csv',
 'XGB_0.793.csv']

In [9]:
['LGBM_0.795.csv','catboost_0.794.csv','XGB_0.793.csv','AutoML_0.78878.csv','keras_0.78326.csv']

['LGBM_0.795.csv',
 'catboost_0.794.csv',
 'XGB_0.793.csv',
 'AutoML_0.78878.csv',
 'keras_0.78326.csv']

In [10]:
best = 0.795
worst = 0.793
w1 = model_weight(0.795, worst, best)
w2 = model_weight(0.794, worst, best) 
w3 = model_weight(0.793, worst, best)
w4 = model_weight(0.788, worst, best)
w5 = model_weight(0.783, worst, best)
#w8 = model_weight(.715, worst, best)


df = pd.read_csv('LGBM_0.795.csv')
df['prediction'] = df['prediction']*w1
preds.append((df['prediction']))
print(np.column_stack(preds).shape)

df2 = pd.read_csv('catboost_0.794.csv')
df2['prediction'] = df2['prediction']*w2
preds.append((df2['prediction']))
print(np.column_stack(preds).shape)

df3 = pd.read_csv('XGB_0.793.csv')
df3['prediction'] = df3['prediction']*w3
preds.append((df3['prediction']))
print(np.column_stack(preds).shape)

"""df4 = pd.read_csv('AutoML_0.78878.csv')
df4['prediction'] = df4['prediction']*w4
preds.append(df4['prediction'])
print(np.column_stack(preds).shape)

df5 = pd.read_csv('keras_0.78326.csv')
df5['prediction'] = df5['prediction']*w5
preds.append(df5['prediction'])
print(np.column_stack(preds).shape)"""



# df8 = pd.read_csv('../input/best-correlated-features-with-low-correlation/submission.csv')
# df8['prediction'] = df8['prediction']*w8
# preds.append(df8['prediction'])
# print(np.column_stack(preds).shape)

(924621, 1)
(924621, 2)
(924621, 3)


"df4 = pd.read_csv('AutoML_0.78878.csv')\ndf4['prediction'] = df4['prediction']*w4\npreds.append(df4['prediction'])\nprint(np.column_stack(preds).shape)\n\ndf5 = pd.read_csv('keras_0.78326.csv')\ndf5['prediction'] = df5['prediction']*w5\npreds.append(df5['prediction'])\nprint(np.column_stack(preds).shape)"

In [11]:
preds

[0        -3.617492
 1        -7.207893
 2        -3.027030
 3        -1.247392
 4         1.883784
             ...   
 924616   -4.242699
 924617    1.249989
 924618   -0.271345
 924619   -0.744686
 924620   -3.183316
 Name: prediction, Length: 924621, dtype: float64,
 0         0.022182
 1         0.001506
 2         0.036879
 3         0.175892
 4         0.783179
             ...   
 924616    0.013792
 924617    0.742194
 924618    0.347119
 924619    0.267197
 924620    0.051176
 Name: prediction, Length: 924621, dtype: float64,
 0         0.001670
 1         0.000405
 2         0.002191
 3         0.033856
 4         0.034988
             ...   
 924616    0.730198
 924617    0.036306
 924618    0.002045
 924619    0.729078
 924620    0.107938
 Name: prediction, Length: 924621, dtype: float64]

In [12]:
%%time
blend_ss = submission.copy()

CPU times: total: 15.6 ms
Wall time: 13.3 ms


In [13]:
blend_ss['prediction'] = (np.sum(np.column_stack(preds), axis=1) / (w1+w2+w3))
blend_ss

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,-1.306778
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,-2.620357
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,-1.086531
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,-0.377325
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.982528
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,-1.272258
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.737633
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.028298
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.091487


In [14]:
os.chdir('C:\\Users\\Khangjrakpam Arjun\\OneDrive - McKinsey & Company\\Documents\\ARJUN\\ML\\COMPETITIONS\\Amex\\amex-default-prediction\\submission\\blended')

In [15]:
blend_ss.to_csv('blended3.csv', index=False)